In [1]:
import sqlite3 as sq
import pandas as pd
import string
import nltk
from nltk.corpus import stopwords
import matplotlib.pyplot as plt


import spacy
from tqdm import tqdm
tqdm.pandas()

# use displacy to visually show the entities. 
from spacy import displacy
import json

In [ ]:
con = sq.connect("NPOPlayer.db")
def run_query(con, query):
    cur = con.cursor()
    cur.execute(query)
    results = cur.fetchall()
    return results

def convert_db_table_to_DF(con, table):
    header_query = "SELECT name FROM pragma_table_info('" + table + "') ORDER BY cid;"
    cols_init = run_query(con, table, header_query)
    cols = [cols_init[i][0] for i in range(len(cols_init))]
    content_query = "SELECT * from " + table
    data = run_query(con, content_query)
    df = pd.DataFrame(data, columns= cols)
    return df

In [ ]:
df = convert_db_table_to_DF(con, "NPOPLAYER")

In [ ]:


# load spacy model. Alternatively you can use en_core_web_lg
nlp = spacy.load("nl_core_news_sm")

In [ ]:
ruler = nlp.add_pipe('entity_ruler', before='ner')

patterns = [
  {"label": "PRODUCT", "pattern": "Wedstrijdanalyse"}
]
ruler.add_patterns(patterns)

In [ ]:
def process(x):
  #print(x[0])
  # there are some pesky NaN in the data. Easy but not so elegant way to fix this.
  if (pd.isna(x[0]) == False and x[0] != ""):
    doc = nlp(x[0])
  elif (pd.isna(x[1]) == False and x[1] != ""):
    doc = nlp(x[1])
  else:
    doc = ""
  return doc

df['plot_entities'] = df[['longSummary', 'shortSummary']].progress_apply(lambda x: process(x), axis = 1)


In [ ]:
#df.to_pickle("NER.pkl")
#df = pd.read_pickle("NER.pkl")


In [ ]:
def get_events(x):
  events = []
  if x != '':
    for entities in x.ents:
      if entities.label_ == 'EVENT':
        events.append(entities.text)
      elif entities.label_ == 'LAW':
        events.append(entities.text)
      elif entities.label_ == 'LOC':
        events.append(entities.text)
      elif entities.label_ == 'GPE':
        events.append(entities.text)
      elif entities.label_ == 'PERSON':
        events.append(entities.text)
      elif entities.label_ == 'PRODUCT':
        events.append(entities.text)
      elif entities.label_ == 'WORK_OF_ART':
        events.append(entities.text)
  return events

# code goes here
df['tags'] = df['plot_entities'].progress_apply(lambda x: get_events(x))
len(df[df["tags"].str.len() == 0])

In [ ]:
media_tag_dict = dict(df[["mediaID", "tags"]].values)


In [ ]:
def save_activities():
    with open('miediaID_NERTags.json', 'w') as outfile:
        json.dump(media_tag_dict, outfile)
save_activities()

Below is to test NER recommendations

In [ ]:
def jaccard_distance(user_ids_isbn_a, user_ids_isbn_b):
                
    set_isbn_a = set(user_ids_isbn_a)
    set_isbn_b = set(user_ids_isbn_b)
    
    union = set_isbn_a.union(set_isbn_b)
    intersection = set_isbn_a.intersection(set_isbn_b)
        
    return len(intersection) / float(len(union))
# code goes here
def get_jaccard_distances(mediaID):
    tags = media_tag_dict[mediaID]
    distancelist = []
    for key, value in media_tag_dict.items():
        distance = jaccard_distance(tags ,value)
        distancelist.append([key, distance])
        
    return distancelist

def get_recommendation(mediaID, topk = 10):
    broadcastExclude = df.query(f"mediaID == '{mediaID}'")["broadcaster"].values[0]
    temp = (pd.DataFrame(get_jaccard_distances(mediaID), columns=['mediaID','JS']).sort_values(by='JS', ascending=False))
    merged = pd.merge(left=temp, right=df, left_on='mediaID', right_on="mediaID", how="inner")
    return merged.query(f"broadcaster != '{broadcastExclude}'").head(topk)

get_recommendation("WO_POWN_8513746")